# Ejercicio

Con base en los datos trabajados en el módulo de muestreo (archivo adjunto en el enlace) extraiga una muestra representativa (calcule el tamaño para un nivel de confianza de 89% y un error de 3% y use ese tamaño específico) 
1. Determine si la clasificación en niveles de desempeño en dos de las cinco competencias genéricas son producto del azar o no.

2. Determine si esos resultados son independientes o dependientes entre sí.

3. Con la muestra, calcule un promedio general por estudiante para las cinco competencias y realice un ranking inicial y un ranking final por departamento; determine si hubo cambios significativos en esa clasificación

[Descargue el archivo aquí](https://www.dropbox.com/s/xgsf3l5gjw0h0q4/saberpro.RData?dl=1)

## Importación de librerías

In [ ]:
library(devtools)
install.packages('coin')
install.packages('plyr')
install.packages('samplingbook')
install_github('DFJL/samplingUtil')
install.packages("magrittr")
install.packages("dplyr")

Loading required package: usethis

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Skipping install of 'SamplingUtil' from a github remote, the SHA1 (e08400ca) has not changed since last install.
  Use `force = TRUE` to force installation

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(devtools)
library(plyr)
library(samplingbook) 
library(SamplingUtil)
library(magrittr) 
library(dplyr) 
library(coin)
library(tidyverse)

Loading required package: pps

Loading required package: sampling

Loading required package: survey

Loading required package: grid

Loading required package: Matrix

Loading required package: survival


Attaching package: ‘survival’


The following objects are masked from ‘package:sampling’:

    cluster, strata



Attaching package: ‘survey’


The following object is masked from ‘package:graphics’:

    dotchart



Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ stringr 1.4.0
✔ tidyr   1.1.1     ✔ forcats 0.5.0
✔ readr   1.3.1     

── Conflicts ────────────────

## Preprocesamiento de datos

### Cargado de los datos desde el link

In [ ]:
load(url('https://www.dropbox.com/s/mp094kybobr2ubs/saberpro.RData?dl=1'))

Warning message in load(url("https://www.dropbox.com/s/mp094kybobr2ubs/saberpro.RData?dl=1")):
“input string 'COLEGIO DE ESTUDIOS SUPERIORES DE ADMINISTRACION-CESA-BOGOTÁ D.C.' cannot be translated to UTF-8, is it valid in 'CP1252'?”
Warning message in load(url("https://www.dropbox.com/s/mp094kybobr2ubs/saberpro.RData?dl=1")):
“input string 'COLEGIO MAYOR DE NUESTRA SEÑORA DEL ROSARIO-BOGOTÁ D.C.' cannot be translated to UTF-8, is it valid in 'CP1252'?”
Warning message in load(url("https://www.dropbox.com/s/mp094kybobr2ubs/saberpro.RData?dl=1")):
“input string 'CORPORACION ESCUELA DE ARTES Y LETRAS-BOGOTÁ D.C.' cannot be translated to UTF-8, is it valid in 'CP1252'?”
Warning message in load(url("https://www.dropbox.com/s/mp094kybobr2ubs/saberpro.RData?dl=1")):
“input string 'CORPORACION INTERNACIONAL PARA EL DESARROLLO EDUCATIVO -CIDE-BOGOTÁ D.C.' cannot be translated to UTF-8, is it valid in 'CP1252'?”
Warning message in load(url("https://www.dropbox.com/s/mp094kybobr2ubs/saberpro.RDa

### Necesitamos eliminar muestras donde hallan datos faltantes en los puntajes



In [ ]:
saber_pro_df <- data.frame(
  index = 1:nrow(sp161718),
  periodo = sp161718$periodo,
  departamento = sp161718$estu_depto_reside,
  raz_cuant_punt = sp161718$raz_cuant_punt,
  raz_cuant_des = sp161718$raz_cuant_des,
  lect_crit_punt = sp161718$lect_crit_punt,
  lect_crit_des = sp161718$lect_crit_des,
  comp_ciud_punt = sp161718$comp_ciud_punt,
  comp_ciud_des = sp161718$comp_ciud_des,
  ingles_punt = sp161718$ingles_punt,
  ingles_desem = sp161718$ingles_desem,
  comu_escr_punt = sp161718$comu_escr_punt,
  comu_escr_des = sp161718$comu_escr_des
)
saber_pro_df <- na.omit(saber_pro_df)

In [ ]:
primer <- substring(saber_pro_df$periodo, 1,4) #A la serie periodo de dejamos sólo el año
saber_pro_df$periodo <- as.character(saber_pro_df$periodo)  #convertimos la serie periodo a caracteres
saber_pro_df$periodo <- primer
saber_pro_df$departamento<-revalue(saber_pro_df$departamento,c('NARIÃƒâ€˜O'="NARIÑO",'NARIÃ‘O'="NARIÑO"))
saber_pro_df$periodo <- as.factor(saber_pro_df$periodo)
saber_pro_df$raz_cuant_des = as.factor(saber_pro_df$raz_cuant_des) 
saber_pro_df$lect_crit_des = as.factor (saber_pro_df$lect_crit_des) 
saber_pro_df$comp_ciud_des = as.factor(saber_pro_df$comp_ciud_des) 
saber_pro_df$ingles_desem = as.factor(saber_pro_df$ingles_desem) 
saber_pro_df$comu_escr_des = as.factor(saber_pro_df$comu_escr_des)

### Realizamos un muestreo aleatorio simple con un error del 3% y nivel de confianza del 89

In [ ]:
#Seleccionamos un tamaño de la muestra con el error 3% y con un nivel de 89%
tamaño_muestra <- sample.size.prop(e=0.03, P = 0.5, N = nrow(saber_pro_df), level = 0.89)$n

indices_muestra_saber_pro <- sample(1:nrow(saber_pro_df),size=tamaño_muestra,replace=FALSE)


In [ ]:
muestra_mas_saber_pro = saber_pro_df[indices_muestra_saber_pro,]

## 1. Determine si la clasificación en niveles de desempeño en dos de las cinco competencias genéricas son producto del azar o no.

### Prueba si la clasificación en **Competencias Ciudadanas** es producto del azar

*Se pueden observar diferencias notables*

In [ ]:
with(chisq.test(table(muestra_mas_saber_pro$comp_ciud_des)), data.frame(observed,expected))

,Var1,Freq,expected
,<fct>,<int>,<dbl>
1,1,192,177.25
2,2,248,177.25
3,3,241,177.25
4,4,28,177.25


In [ ]:
chisq.test(table(muestra_mas_saber_pro$comp_ciud_des))


	Chi-squared test for given probabilities

data:  table(muestra_mas_saber_pro$comp_ciud_des)
X-squared = 178.07, df = 3, p-value < 2.2e-16


### *La prueba de bondad de ajuste o $\chi^2$ test muestra que el p-valor da mucho menor que nuestro error de 3%. Es decir **SE RECHAZA** la hipótesis nula y se concluye que los datos no son producto del azar*



### Prueba si la clasificación en **Comunicación Escrita** es producto del azar

*Se pueden observar diferencias notables*

In [ ]:
with(chisq.test(table(muestra_mas_saber_pro$comu_escr_des)), data.frame(observed,expected))

,Var1,Freq,expected
,<fct>,<int>,<dbl>
1,1,88,177.25
2,2,270,177.25
3,3,249,177.25
4,4,102,177.25


In [ ]:
chisq.test(table(muestra_mas_saber_pro$comu_escr_des))


	Chi-squared test for given probabilities

data:  table(muestra_mas_saber_pro$comu_escr_des)
X-squared = 154.46, df = 3, p-value < 2.2e-16


### *La prueba de bondad de ajuste o $\chi^2$ test muestra que el p-valor da mucho menor que nuestro error de 3%. Es decir **SE RECHAZA** la hipótesis nula y se concluye que los datos no son producto del azar*



## 2. Determine si esos resultados son independientes o dependientes entre sí.

### Prueba si la clasificación en competencias ciudadanas y comunicación escrita son independientes

In [ ]:
with(muestra_mas_saber_pro,table(comu_escr_des, comp_ciud_des))

             comp_ciud_des
comu_escr_des   1   2   3   4
            1  33  40  13   2
            2  76 104  84   6
            3  66  77  95  11
            4  17  27  49   9

In [ ]:
with(muestra_mas_saber_pro,chisq.test(table(comu_escr_des, comp_ciud_des)))

Warning message in chisq.test(table(comu_escr_des, comp_ciud_des)):
“Chi-squared approximation may be incorrect”



	Pearson's Chi-squared test

data:  table(comu_escr_des, comp_ciud_des)
X-squared = 41.118, df = 9, p-value = 4.761e-06


### *La prueba de independencia nos muestra que el p-valor es mucho menor que nuestro error de 3%. Es decir **SE RECHAZA** la hipótesis nula y concluimos que que las dos clasificaciones son independientes entre si.*



### Prueba si la clasificación en razonamiento cuantitativo e inglés son independientes

In [ ]:
with(muestra_mas_saber_pro,table(raz_cuant_des, ingles_desem))

             ingles_desem
raz_cuant_des A1 A1- A2 B1 B2
            1 69  61 19 12  2
            2 98  53 59 19  7
            3 61  18 86 87 32
            4  2   1  4  9 10

In [ ]:
with(muestra_mas_saber_pro,chisq.test(table(comu_escr_des, comp_ciud_des)))

Warning message in chisq.test(table(comu_escr_des, comp_ciud_des)):
“Chi-squared approximation may be incorrect”



	Pearson's Chi-squared test

data:  table(comu_escr_des, comp_ciud_des)
X-squared = 41.118, df = 9, p-value = 4.761e-06


### *La prueba de independencia nos muestra que el p-valor es mucho menor que nuestro error de 3%. Es decir **SE RECHAZA** la hipótesis nula y concluimos que que las dos clasificaciones **SON DEPENDIENTES** entre si.*



## 3. Con la muestra, calcule un promedio general por estudiante para las cinco competencias y realice un ranking inicial y un ranking final por departamento; determine si hubo cambios significativos en esa clasificación

### Realizamos una agrupación por año y por departamento y aplicamos el promedio entre los puntajes

In [ ]:
muestra_puntaje_promedio <- data.frame(
  periodo = saber_pro_df$periodo,
  departamento = saber_pro_df$departamento,
  promedio = with(saber_pro_df, (comp_ciud_punt+comu_escr_punt+ingles_punt+lect_crit_punt+raz_cuant_punt)/5)
)

promedio_dptos_año <- muestra_puntaje_promedio %>%
  select(departamento,periodo,promedio) %>%
  group_by(departamento,periodo) %>%
  summarise(puntaje_dpto=mean(promedio))

`summarise()` regrouping output by 'departamento' (override with `.groups` argument)



### Separamos entre cada periodo

In [ ]:
deptos_2017<-promedio_dptos_año[promedio_dptos_año$periodo=='2017',c('departamento','puntaje_dpto')]
deptos_2018<-promedio_dptos_año[promedio_dptos_año$periodo=='2018',c('departamento','puntaje_dpto')]

### Realizamos un ordenamiento por el puntaje, asignamos una clasficacióón y reordenamos por departamento para comparar los puestos de cada departamento

In [ ]:
ordenados_deptos_2017<-deptos_2017 %>% arrange(desc(puntaje_dpto))
ordenados_deptos_2018<-deptos_2018 %>% arrange(desc(puntaje_dpto))

ordenados_deptos_2017$puesto  <- 1:nrow(ordenados_deptos_2017)
ordenados_deptos_2018$puesto  <- 1:nrow(ordenados_deptos_2018)

ordenados_deptos_2017 <- ordenados_deptos_2017[order(ordenados_deptos_2017$departamento),]
ordenados_deptos_2018 <- ordenados_deptos_2018[order(ordenados_deptos_2018$departamento),]

In [ ]:
wilcoxsign_test(ordenados_deptos_2017$puesto ~ ordenados_deptos_2018$puesto,  distribution = "exact")


	Exact Wilcoxon-Pratt Signed-Rank Test

data:  y by x (pos, neg) 
	 stratified by block
Z = 1.1865, p-value = 0.241
alternative hypothesis: true mu is not equal to 0


### *La prueba de signos nos muestra que el p-valor es mucho mayor que nuestro error de 3%. Es decir **NO SE RECHAZA** la hipóótesis nula, por lo tanto podemos concluir que no hubo cambios en la clasificación por departamentos entre 2017 y 2018*

